%md

## Jobs
W sekcji **Jobs** znajdziemy listę wszystkich zadań (jobs) uruchomionych w ramach aplikacji Spark. Dla każdego joba dostępne są informacje:
- Status wykonania (np. succeeded, failed)
- Czas trwania joba
- Powiązane stages
- Liczba tasków i ich przebieg

## Stages
Sekcja **Stages** pokazuje podział joba na etapy (stages), które są tworzone na podstawie granic przetasowania danych (shuffle boundaries). Można tu zobaczyć:
- Numery i nazwy stages
- Ilość i stan tasków
- Statystyki wykonania (czas, ilość przetworzonych danych, wejście/wyjście)
- Szczegółowy podgląd każdego taska

## Storage
W zakładce **Storage** znajdują się informacje o zcache’owanych (persistowanych) zbiorach danych:
- Lista RDD/DataFrame, które są przechowywane w pamięci
- Rozmiar danych w pamięci i na dysku
- Liczba partycji i ich rozmieszczenie

## Executors
Sekcja **Executors** pokazuje informacje o wszystkich executorach w klastrze:
- Ilość dostępnej i użytej pamięci
- Ilość wykonanych tasków
- Czas działania
- Ilość odczytanych i zapisanych danych
- Pomaga w analizie rozproszenia i wydajności przetwarzania

## SQL/DataFrame
Zakładka **SQL/DataFrame** zawiera szczegóły zapytań SQL i operacji DataFrame:
- Fazy planowania zapytania (parsed, analyzed, optimized, physical plan)
- Informacje o wykonaniu zapytań
- Możliwość przeglądu DAG (Directed Acyclic Graph) zapytania
- Statystyki dotyczące czasu i zasobów

## Gdzie jest informacja o dystrybucji danych?
Informację o dystrybucji danych w Spark można znaleźć głównie w sekcjach:
- **Stages** – rozkład tasków na partycje
- **Storage** – rozmieszczenie danych w partycjach
- **Executors** – alokacja partycji na konkretne executory oraz ich zużycie zasobów

In [0]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("Bucketing vs Partitioning").getOrCreate()

filePath = "dbfs:/FileStore/tables/Files/names.csv"

df = spark.read.option("header", "true").option("inferSchema", "true").csv(filePath)
df.printSchema()

partitioned_path = "/tmp/names_partitioned"
df.write.mode("overwrite").partitionBy("place_of_birth").parquet(partitioned_path)

print("Partitioned table files:")
display(dbutils.fs.ls(partitioned_path))

spark.conf.set("spark.sql.catalogImplementation", "hive")
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true")

bucketed_table = "names_bucketed"

df.write.mode("overwrite") \
    .format("parquet") \
    .bucketBy(4, "place_of_birth") \
    .sortBy("name") \
    .saveAsTable(bucketed_table)

bucketed_table_path = spark.sql(f"DESCRIBE FORMATTED {bucketed_table}") \
    .filter("col_name = 'Location'") \
    .select("data_type") \
    .collect()[0][0]

print("Bucketed table files:")
display(dbutils.fs.ls(bucketed_table_path))


root
 |-- imdb_name_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_name: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- bio: string (nullable = true)
 |-- birth_details: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- place_of_birth: string (nullable = true)
 |-- death_details: string (nullable = true)
 |-- date_of_death: string (nullable = true)
 |-- place_of_death: string (nullable = true)
 |-- reason_of_death: string (nullable = true)
 |-- spouses_string: string (nullable = true)
 |-- spouses: integer (nullable = true)
 |-- divorces: integer (nullable = true)
 |-- spouses_with_children: integer (nullable = true)
 |-- children: integer (nullable = true)

